## **09-03 영어/한국어 Word2Vec 실습**
---

https://wikidocs.net/50739

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [ ]:
## colab에서 dependency 문제로 버전 세팅
!pip uninstall numpy gensim tensorflow numba -y
!pip install numpy==1.26.4
!pip install gensim==4.3.3 tensorflow==2.18.0 numba==0.60.0

In [ ]:
# 런타임 자동으로 재시작 - NumPy와 C-extension 기반 라이브러리(gensim 등)의 바이너리 불일치 문제 해결
import os
os.kill(os.getpid(), 9)

In [ ]:
import numpy
import gensim
import tensorflow as tf

print("numpy version:", numpy.__version__)
print("gensim version:", gensim.__version__)
print("tensorflow version:", tf.__version__)


In [ ]:
gensim.__version__

---
### **1. 영어 Word2Vec 만들기**

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [ ]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

#### **1) 훈련 데이터 이해하기**
훈련 데이터를 다운로드 합니다.

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

#### **2) 훈련 데이터 전처리하기**
위 데이터를 위한 전처리 코드는 아래와 같습니다.

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))
print('첫 번째 문장 : {}'.format(result[0])) # 첫 번째 문장

총 샘플의 개수는 약 27만 3천개입니다.

In [ ]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

#### **3) Word2Vec 훈련시키기**

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

Word2Vec의 하이퍼파라미터값은 다음과 같습니다.

**vector_size** = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
**window** = 컨텍스트 윈도우 크기  
**min_count** = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
**workers** = 학습을 위한 프로세스 수  
**sg** = 0은 CBOW, 1은 Skip-gram.  

Word2Vec에 대해서 학습을 진행하였습니다. Word2Vec는 입력한 단어에 대해서 가장 유사한 단어들을 출력하는 model.wv.most_similar을 지원합니다. man과 가장 유사한 단어들은 어떤 단어들일까요?

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

man과 유사한 단어로 woman, guy, boy, lady, girl, gentleman, soldier, kid 등을 출력하는 것을 볼 수 있습니다. Word2Vec를 통해 단어의 유사도를 계산할 수 있게 되었습니다.

#### **4) Word2Vec 모델 저장하고 로드하기**
공들여 학습한 모델을 언제든 나중에 다시 사용할 수 있도록 컴퓨터 파일로 저장하고 다시 로드해보겠습니다. 이 모델을 가지고 향후 시각화를 진행할 예정이므로 꼭 저장해주세요.

In [ ]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

로드한 모델에 대해서 다시 man과 유사한 단어를 출력해보겠습니다.

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

---
### **2. 한국어 Word2Vec 만들기(네이버 영화 리뷰)**

네이버 영화 리뷰 데이터로 한국어 Word2Vec을 만들어봅시다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

네이버 영화 리뷰 데이터를 다운로드합니다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

네이버 영화 리뷰 데이터를 데이터프레임으로 로드하고 상위 5개의 행을 출력해봅시다.

In [ ]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data[:5] # 상위 5개 출력

총 리뷰 개수를 확인해보겠습니다.

In [ ]:
print('리뷰 개수 :', len(train_data))

총 20만개의 샘플이 존재하는데, 결측값 유무를 확인합니다.

In [ ]:
print('NULL 값 존재 유무 :', train_data.isnull().values.any())

In [ ]:
train_data.isnull().sum() # NULL 값 개수 확인

결측값이 존재하므로 결측값이 존재하는 행을 제거합니다.

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('NULL 값 존재 유무 :', train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
train_data.isnull().sum() # NULL 값 개수 확인

결측값이 삭제된 후의 리뷰 개수를 확인합니다.

In [ ]:
print('리뷰 개수 :', len(train_data))

총 199,992개의 리뷰가 존재합니다. 정규 표현식을 통해 한글이 아닌 경우 제거하는 전처리를 진행합니다.

In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data.shape

In [ ]:
train_data[:5] # 상위 5개 출력

학습 시에 사용하고 싶지 않은 단어들인 불용어를 제거하겠습니다. 형태소 분석기 Okt를 사용하여 각 문장에 대해서 일종의 단어 내지는 형태소 단위로 나누는 토큰화를 수행합니다. 다소 시간이 소요될 수 있습니다.

In [ ]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를',
             '으로','자','에','와','한','하다']

In [ ]:
%%time
# CPU times: total: 11min 37s
# Wall time: 14min 43s

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
# colab gpu에서 10분 이상 소요됨
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화 (어간추출)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
print(tokenized_data[:3])
train_data['document'][:3]

토큰화가 된 상태에서는 각 리뷰의 길이 분포 또한 확인이 가능합니다.

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :', max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :', sum(map(len, tokenized_data)) / len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

**단어 임베딩**  
  
Word2Vec으로 토큰화 된 네이버 영화 리뷰 데이터를 학습합니다.

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(
    sentences=tokenized_data,  # 학습에 사용할 말뭉치 (토큰화된 문장 리스트)
    vector_size=100,           # 각 단어를 표현할 벡터의 차원 수 (예: 100차원 벡터)
    window=5,                  # 현재 단어를 기준으로 좌우로 고려할 최대 거리 (윈도우 크기)
    min_count=5,               # 최소 등장 빈도 수. 이보다 적게 등장한 단어는 무시됨
    workers=4,                 # 학습에 사용할 CPU 코어 수 (병렬처리용)
    sg=0                       # 학습 알고리즘 선택: 0=CBOW, 1=Skip-gram
)

In [ ]:
print('완성된 임베딩 매트릭스의 크기 확인 :', model.wv.vectors.shape)

학습이 다 되었다면 Word2Vec 임베딩 행렬의 크기를 확인합니다.

총 17,615개의 단어가 존재하며 각 단어는 100차원으로 구성되어져 있습니다. '최민식'과 유사한 단어들을 뽑아봅시다.

In [ ]:
print(model.wv.most_similar("최민식"))

'히어로'와 유사한 단어들을 뽑아봅시다.

In [ ]:
print(model.wv.most_similar("히어로"))

In [ ]:
print(model.wv.most_similar("발연기"))

In [ ]:
model.wv.similar_by_word("배트맨")

---
### **3. 사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding) 소개**
자연어 처리 작업을 할때, **케라스의 Embedding()** 를 사용하여 갖고 있는 훈련 데이터로부터 처음부터 임베딩 벡터를 훈련시키기도 하지만, 위키피디아 등의 방대한 데이터로 **사전에 훈련된 워드 임베딩(pre-trained word embedding vector)** 를 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용 할 수도 있습니다.

예를 들어서 감성 분류 작업을 하는데 훈련 데이터의 양이 부족한 상황이라면, 다른 방대한 데이터를 `Word2Vec`이나 `GloVe` 등으로 **사전에 학습시켜놓은 임베딩 벡터** 들을 가지고 와서 모델의 입력으로 사용하는 것이 때로는 더 좋은 성능을 얻을 수 있습니다. 사전 훈련된 워드 임베딩을 가져와서 간단히 단어들의 유사도를 구해보는 실습을 해보겠습니다. 실제로 모델에 적용해보는 실습은 향후에 진행합니다.

**구글이 제공하는 사전 훈련된(미리 학습되어져 있는) Word2Vec 모델** 을 사용하는 방법에 대해서 알아보겠습니다. 구글은 사전 훈련된 3백만 개의 Word2Vec 단어 벡터들을 제공합니다. 각 **임베딩 벡터의 차원은 300** 입니다. gensim을 통해서 이 모델을 불러오는 건 매우 간단합니다. 이 모델을 다운로드하고 파일 경로를 기재하면 됩니다.

모델 다운로드 경로 (GoogleNews-vectors-negative300.bin.gz) : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

압축 파일의 용량은 약 1.5GB이지만, 파일의 압축을 풀면 약 3.3GB의 파일이 나옵니다.

In [ ]:
# import gensim
# import urllib.request

# # 구글의 사전 훈련된 Word2Vec 모델을 로드.
# urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
#                            filename="GoogleNews-vectors-negative300.bin.gz")
# word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
# gensim downloader로 Google News Word2Vec 모델 로드
from gensim.downloader import load

# 약 1.5GB 모델 자동 다운로드 (최초 1회만 다운로드됨)
print("모델 다운로드 중...")
word2vec_model = load("word2vec-google-news-300")
print("모델 로딩 완료!")

In [ ]:
import gensim

모델의 크기(shape)를 확인해봅시다.

In [ ]:
# 예시: 단어 벡터 확인
print(word2vec_model['computer'])
word2vec_model['computer'].shape

In [ ]:
print(word2vec_model.vectors.shape)

모델의 크기는 3,000,000 x 300입니다. 즉, 3백만 개의 단어와 각 단어의 차원은 300입니다. 파일의 크기가 3기가가 넘는 이유를 계산해보면 아래와 같습니다.

* 3 million words * 300 features * 4bytes/feature = ~3.35GB  

사전 훈련된 임베딩을 사용하여 두 단어의 유사도를 계산해봅시다.

In [ ]:
print(word2vec_model.similarity('this', 'is'))
print(word2vec_model.similarity('post', 'book'))

단어 'book'의 벡터를 출력해봅시다.

In [ ]:
print(word2vec_model['book'])

참고 : Word2vec 모델은 자연어 처리에서 단어를 밀집 벡터로 만들어주는 단어 임베딩 방법론이지만 최근에 들어서는 자연어 처리를 넘어서 추천 시스템에도 사용되고 있는 모델입니다. 적당하게 데이터를 나열해주면 Word2vec은 위치가 근접한 데이터를 유사도가 높은 벡터를 만들어준다는 점에서 착안된 아이디어입니다. 관심있는 분들은 구글에 'item2vec'을 검색해보세요.